<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/jvcqp2iy2jlx2b32rmzdt0tx8lvxgzkp.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>RECURRENT NETWORKS and LSTM IN DEEP LEARNING</font></h1>


## Applying Recurrent Neural Networks/LSTM for Language Modelling
Hello and welcome to this part. In this notebook, we will go over the topic of what Language Modelling is and create a Recurrent Neural Network model based on the Long Short-Term Memory unit to train and be benchmarked by the Penn Treebank. By the end of this notebook, you should be able to understand how TensorFlow builds and executes a RNN model for Language Modelling.


--------

## The Objective
By now, you should have an understanding of how Recurrent Networks work -- a specialized model to process sequential data by keeping track of the "state" or context. In this notebook, we go over a TensorFlow code snippet for creating a model focused on **Language Modelling** -- a very relevant task that is the cornerstone of many different linguistic problems such as **Speech Recognition, Machine Translation and Image Captioning**. For this, we will be using the Penn Treebank, which is an often-used dataset for benchmarking Language Modelling models.

## What exactly is Language Modelling?
Language Modelling, to put it simply, **is the task of assigning probabilities to sequences of words**. This means that, given a context of one or a few words in the language the model was trained on, the model should have a knowledge of what are the most probable words or sequence of words for the sentence. Language Modelling is one of the tasks under Natural Language Processing, and one of the most important.

<img src=https://ibm.box.com/shared/static/1d1i5gub6wljby2vani2vzxp0xsph702.png width="768"/>
<center>*Example of a sentence being predicted*</center>

In this example, one can see the predictions for the next word of a sentence, given the context "This is an". As you can see, this boils down to a sequential data analysis task -- you are given a word or a sequence of words (the input data), and, given the context (the state), you need to find out what is the next word (the prediction). This kind of analysis is very important for language-related tasks such as **Speech Recognition, Machine Translation, Image Captioning, Text Correction** and many other very relevant problems. 

<img src=https://ibm.box.com/shared/static/az39idf9ipfdpc5ugifpgxnydelhyf3i.png width="1080"/>
<center>*The above example schematized as an RNN in execution*</center>

As the above image shows, Recurrent Network models fit this problem like a glove. Alongside LSTM and its capacity to maintain the model's state for over one thousand time steps, we have all the tools we need to undertake this problem. The goal for this notebook is to create a model that can reach **low levels of perplexity** on our desired dataset.

For Language Modelling problems, **perplexity** is the way to gauge efficiency. Perplexity is simply a measure of how well a probabilistic model is able to predict its sample. A higher-level way to explain this would be saying that **low perplexity means a higher degree of trust in the predictions the model makes**. Therefore, the lower perplexity is, the better.

## The Penn Treebank dataset
Historically, datasets big enough for Natural Language Processing are hard to come by. This is in part due to the necessity of the sentences to be broken down and tagged with a certain degree of correctness -- or else the models trained on it won't be able to be correct at all. This means that we need a **large amount of data, annotated by or at least corrected by humans**. This is, of course, not an easy task at all.

The Penn Treebank, or PTB for short, is a dataset maintained by the University of Pennsylvania. It is *huge* -- there are over **four million and eight hundred thousand** annotated words in it, all corrected by humans. It is composed of many different sources, from abstracts of Department of Energy papers to texts from the Library of America. Since it is verifiably correct and of such a huge size, the Penn Treebank has been used time and time again as a benchmark dataset for Language Modelling.

The dataset is divided in different kinds of annotations, such as Piece-of-Speech, Syntactic and Semantic skeletons. For this example, we will simply use a sample of clean, non-annotated words (with the exception of one tag -- `<unk>`, which is used for rare words such as uncommon proper nouns) for our model. This means that we just want to predict what the next words would be, not what they mean in context or their classes on a given sentence. 
<br/>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<center>the percentage of lung cancer deaths among the workers at the west `<unk>` mass. paper factory appears to be the highest for any asbestos workers studied in western industrialized countries he said 
 the plant which is owned by `<unk>` & `<unk>` co. was under contract with `<unk>` to make the cigarette filters 
 the finding probably will support those who argue that the u.s. should regulate the class of asbestos including `<unk>` more `<unk>` than the common kind of asbestos `<unk>` found in most schools and other buildings dr. `<unk>` said</center>
</div>
 <center>*Example of text from the dataset we are going to use, `ptb.train`*</center>
 <br/>



<h2>Word Embeddings</h2><br/>

For better processing, in this example, we will make use of [**word embeddings**]( [https://www.tensorflow.org/tutorials/word2vec/), which are **a way of representing sentence structures or words as n-dimensional vectors (where n is a reasonably high number, such as 200 or 500) of real numbers**. Basically, we will assign each word a randomly-initialized vector, and input those into the network to be processed. After a number of iterations, these vectors are expected to assume values that help the network to correctly predict what it needs to -- in our case, the probable next word in the sentence. This is shown to be very effective in Natural Language Processing tasks, and is a commonplace practice.
<br/><br/>
<font size = 4><strong>
$$Vec("Example") = [0.02, 0.00, 0.00, 0.92, 0.30,...]$$
</font></strong>
<br/>
Word Embedding tends to group up similarly used words *reasonably* together in the vectorial space. For example, if we use T-SNE (a dimensional reduction visualization algorithm) to flatten the dimensions of our vectors into a 2-dimensional space and use the words these vectors represent as their labels, we might see something like this:

<img src=https://ibm.box.com/shared/static/bqhc5dg879gcoabzhxra1w8rkg3od1cu.png width="800"/>
<center>*T-SNE Mockup with clusters marked for easier visualization*</center>

As you can see, words that are frequently used together, in place of each other, or in the same places as them tend to be grouped together -- being closer together the higher these correlations are. For example, "None" is pretty semantically close to "Zero", while a phrase that uses "Italy" can probably also fit "Germany" in it, with little damage to the sentence structure. A vectorial "closeness" for similar words like this is a great indicator of a well-built model.

---
 

We need to import the necessary modules for our code. We need **`numpy` and `tensorflow`**, obviously. Additionally, we can import directly the **`tensorflow.models.rnn.rnn`** model, which includes the function for building RNNs, and **`tensorflow.models.rnn.ptb.reader`** which is the helper module for getting the input data from the dataset we just downloaded.

If you want to learm more take a look at https://www.tensorflow.org/versions/r0.11/api_docs/python/rnn_cell/

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import tensorflow as tf
import os

print('TensorFlow version: ', tf.__version__)

TensorFlow version:  1.1.0


In [2]:
tf.reset_default_graph()

In [3]:
if not os.path.isfile('./penn_treebank_reader.py'):
    print('Downloading penn_treebank_reader.py...')
    !wget -q -O ../data/Penn_Treebank/ptb.zip https://ibm.box.com/shared/static/z2yvmhbskc45xd2a9a4kkn6hg4g4kj5r.zip
    !unzip -o ../data/Penn_Treebank/ptb.zip -d ../data/Penn_Treebank
    !cp ../data/Penn_Treebank/ptb/reader.py ./penn_treebank_reader.py
else:
    print('Using local penn_treebank_reader.py...')

Using local penn_treebank_reader.py...


In [4]:
import penn_treebank_reader as reader

## Building the LSTM model for Language Modeling
Now that we know exactly what we are doing, we can start building our model using TensorFlow. The very first thing we need to do is download and extract the `simple-examples` dataset, which can be done by executing the code cell below.

In [5]:
if not os.path.isfile('../data/Penn_Treebank/simple_examples.tgz'):
    !wget -O ../data/Penn_Treebank/simple_examples.tgz http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
    !tar xzf ../data/Penn_Treebank/simple_examples.tgz -C ../data/Penn_Treebank/

Additionally, for the sake of making it easy to play around with the model's hyperparameters, we can declare them beforehand. Feel free to change these -- you will see a difference in performance each time you change those!  



In [6]:
#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size = 200
#The maximum number of epochs trained with the initial learning rate
max_epoch = 4
#The total number of epochs in training
max_max_epoch = 13
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 30
#The size of our vocabulary
vocab_size = 10000
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "../data/Penn_Treebank/simple-examples/data/"

Some clarifications for LSTM architecture based on the argumants:

Network structure:
- In this network, the number of LSTM cells are 2. To give the model more expressive power, we can add multiple layers of LSTMs to process the data. The output of the first layer will become the input of the second and so on.
- The recurrence steps is 20, that is, when our RNN is "Unfolded", the recurrence step is 20.   
- the structure is like: 
     - 200 input units -> [200x200] Weight -> 200 Hidden units (first layer) -> [200x200] Weight matrix  -> 200 Hidden units (second layer) ->  [200] weight Matrix -> 200 unit output

Hidden layer:
- Each LSTM has 200 hidden units which is equivalant to the dimensianality of the embedding words and output. 

Input layer: 
- The network has 200 input units. 
- Suppose each word is represented by an embedding vector of dimensionality e=200. The input layer of each cell will have 200 linear units. These e=200 linear units are connected to each of the h=200 LSTM units in the hidden layer (assuming there is only one hidden layer, though our case has 2 layers).
- The input shape is [batch_size, num_steps], that is [30x20]. It will turn into [30x20x200] after embedding, and then 20x[30x200]




There is a lot to be done and a ton of information to process at the same time, so go over this code slowly. It may seem complex at first, but if you try to ally what you just learned about language modelling to the code you see, you should be able to understand it.

This code is adapted from the [PTBModel](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/rnn/ptb/ptb_word_lm.py) example bundled with the TensorFlow source code.


#### Train data
The story starts from data: 
- Train data is a list of words, represented by numbers - N=929589 numbers, e.g. [9971, 9972, 9974, 9975,...]
- We read data as mini-batch of size b=30. Assume the size of each sentence is 20 words (num_steps = 20). Then it will take int(N/b*h)+1=1548 iterations for the learner to go through all sentences once. So, the number of iterators is 1548
- Each batch data is read from train dataset of size 600, and shape of [30x20]

First we start an interactive session:

In [7]:
session=tf.InteractiveSession()

In [8]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _ = raw_data

Lets just read one mini-batch now and feed our network:

In [9]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple=next(itera)
x=first_touple[0]
y=first_touple[1]

In [10]:
x.shape

(30, 20)

Lets look at 3 sentences of our input x:

In [11]:
x[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21]], dtype=int32)

In [12]:
size = hidden_size

we define 2 place holders to feed them with mini-batchs, that is x and y:

In [13]:
_input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
_targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

lets defin a dictionary, and use it later to feed the placeholders with our first mini-batch:

In [14]:
feed_dict={_input_data:x, _targets:y}

For example, we can use it to feed _input_data:

In [15]:
session.run(_input_data,feed_dict)

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21],
       [   3,   71,    4,   27,  246,   60,   11,  215,    4,    1, 1846,
           9,    3,   71,  546,    2, 6505,  162,    6,  104],
       [  93,   25,    6,  261,  681,  251,    0,  278, 3246,   13,  200,
           1,    8,  105, 3360,    1,    4,    0,  536,    4],
       [  20,    6,  954,   12,    3,   21,   78,   14,  977,  726,    0,
          37,   42,   34,    5,  437,  116,  206,  927,    2],
       [  18,  296,    7,  201,   76,    4,  182,  560, 3836,   17,  974,
         975,    6,  942,    4,  156,   77, 1570,  288,  644],
       [  23, 1238,  899,    5,   25,  20

In this step, we create the stacked LSTM, which is a 2 layer LSTM network:

In [16]:
stacked_lstm = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0) 
                                            for _ in range(num_layers)]
                                          )

Also, we initialize the states of the nework:

#### _initial_state

For each LCTM, there are 2 state matrics, c_state and m_state.  c_state and m_state represent "Memory State" and "Cell State". Each hidden layer, has a vector of size 30, which keeps the states. so, for 200 hidden units in each LSTM, we have a matrix of size [30x200]

In [17]:
_initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
_initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(30, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(30, 200) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(30, 200) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(30, 200) dtype=float32>))

lets look at the states, though they are all zero for now:

In [18]:
session.run(_initial_state,feed_dict)

(LSTMStateTuple(c=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), h=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)),
 LSTMStateTuple(c=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), h=array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., 

### Embeddings
We create the embeddings for our input data. embedding is dictionary of [10000x200] for all 10000 unique words.

In [19]:
try:
    embedding = tf.get_variable("embedding", [vocab_size, hidden_size])  #[10000x200]
except ValueError:
    pass
embedding.get_shape().as_list()

[10000, 200]

In [20]:
session.run(tf.global_variables_initializer())
session.run(embedding, feed_dict)

array([[-0.01209606, -0.00610176,  0.0101538 , ...,  0.0226196 ,
         0.00843582,  0.02089332],
       [-0.00645194,  0.01049244,  0.02086505, ...,  0.00153445,
         0.02180625, -0.00136558],
       [ 0.00315042,  0.02083927,  0.02305393, ..., -0.01826529,
        -0.00958337, -0.01164767],
       ..., 
       [ 0.01118658, -0.02081927,  0.01142085, ...,  0.02072189,
        -0.01370281, -0.01318221],
       [ 0.01070726,  0.0197452 , -0.01347473, ..., -0.0122654 ,
        -0.00089327,  0.01057745],
       [ 0.0126599 ,  0.01401849, -0.01375013, ..., -0.01807761,
         0.01567172, -0.01887284]], dtype=float32)

embedding_lookup goes to each row of input_data, and for each word in the row/sentence, finds the correspond vector in embedding.
It creates a [30*20*200] matrix, so, the first elemnt of __inputs__ (the first sentence), is a matrix of 20x200, which each row of it is vector representing a word in the sentence.


In [21]:
# Define where to get the data for our embeddings from
inputs = tf.nn.embedding_lookup(embedding, _input_data)  #shape=(30, 20, 200) 

In [22]:
inputs

<tf.Tensor 'embedding_lookup:0' shape=(30, 20, 200) dtype=float32>

In [23]:
session.run(inputs[0], feed_dict)

array([[-0.00346966,  0.0180132 , -0.0067113 , ..., -0.02043601,
         0.01268665, -0.01841335],
       [ 0.01141287, -0.01209113,  0.00185063, ..., -0.00534211,
         0.00288254, -0.00211206],
       [-0.00873637,  0.01713076,  0.0013929 , ...,  0.00410479,
         0.01605235, -0.00925095],
       ..., 
       [ 0.02404756,  0.00544174, -0.02302063, ...,  0.02057038,
         0.01565341,  0.00142729],
       [-0.01898457, -0.01439425,  0.01303636, ...,  0.01108175,
        -0.0069335 ,  0.01917717],
       [-0.02178566,  0.00721733, -0.00448469, ..., -0.00281648,
        -0.01761998,  0.01708188]], dtype=float32)

### Constructing Recurrent Neural Networks
__tf.nn.dynamicrnn()__ creates a recurrent neural network using __stacked_lstm__ which is an instance of RNNCell. 

The input should be a Tensor of shape: [batch_size, max_time, ...], in our case it would be (30, 20, 200)

This method, returns a pair (outputs, new_state) where:
- outputs is a length T list of outputs (one for each input), or a nested tuple of such elements.
- new_state is the final state


In [24]:
outputs, new_state =  tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=_initial_state)

so, lets look at the outputs. The output of the stackedLSTM comes from 200 hidden_layer, and in each time step(=20), one of them get activated. we use the linear activation to map the 200 hidden layer to a [?x10 matrix]

In [25]:
outputs

<tf.Tensor 'rnn/transpose:0' shape=(30, 20, 200) dtype=float32>

In [26]:
session.run(tf.global_variables_initializer())
session.run(outputs[0], feed_dict)

array([[ -1.20562829e-04,   6.02060099e-05,   2.73827434e-04, ...,
          1.54950991e-04,   2.65133771e-04,   4.74843808e-04],
       [  8.81994274e-05,  -1.02157450e-04,   3.84475599e-04, ...,
          2.33782237e-04,   4.26791579e-04,   3.77696066e-04],
       [  1.94964101e-04,   3.63061205e-04,   3.34488461e-04, ...,
          2.31641243e-04,   5.73628291e-04,   2.21637674e-04],
       ..., 
       [  4.45521146e-04,  -1.39987678e-03,  -3.23443557e-04, ...,
          2.83046276e-04,  -6.50624046e-04,  -6.72494993e-04],
       [  1.38641539e-04,  -1.15363649e-03,  -3.51849681e-04, ...,
          6.35754317e-04,  -6.62986364e-04,   1.25905761e-04],
       [  1.03857907e-04,  -3.09593888e-04,  -4.67124657e-04, ...,
         -1.76438043e-05,  -7.08129141e-04,   6.73512812e-04]], dtype=float32)

Lets reshape the output tensor from  [30 x 20 x 200] to [600 x 200]

In [27]:
output = tf.reshape(outputs, [-1, size])
output

<tf.Tensor 'Reshape:0' shape=(600, 200) dtype=float32>

In [28]:
session.run(output[0], feed_dict)

array([ -1.20562829e-04,   6.02060099e-05,   2.73827434e-04,
         2.16999479e-06,  -3.37485872e-05,   9.76368319e-05,
        -7.20815151e-05,   3.59538681e-04,   9.69108951e-05,
        -4.51735919e-04,  -1.10498120e-04,  -2.26888660e-04,
        -1.21247562e-04,   4.21536941e-04,   4.02809412e-04,
        -4.73341133e-05,   2.61181180e-04,  -1.46595950e-04,
         4.20109078e-04,   8.05012300e-04,  -2.60946726e-05,
         1.66126789e-04,  -2.78372201e-04,   3.64558015e-04,
         2.12733750e-04,  -9.33654446e-05,   3.72408860e-04,
        -1.01597579e-04,  -1.61287506e-04,  -2.98267434e-04,
         2.69479002e-04,   3.75117015e-05,   8.22367219e-05,
        -2.49352306e-04,  -4.92326973e-04,   2.37040411e-04,
         2.16158223e-04,  -2.16564767e-05,  -2.29183483e-04,
        -6.79773861e-04,  -2.80815468e-04,  -5.21420094e-04,
         5.34663341e-05,  -8.46006849e-04,  -2.81927551e-05,
         1.32858346e-04,   6.61207287e-06,   9.46656437e-05,
         4.75724839e-04,

### logistic unit
Now, we create a logistic unit to return the probability of the output word. That is, mapping the 600

Softmax = [600 x 200]* [200 x 1000]+ [1 x 1000] -> [600 x 1000]

In [29]:
softmax_w = tf.get_variable("softmax_w", [size, vocab_size]) #[200x1000]
softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
logits = tf.matmul(output, softmax_w) + softmax_b

In [30]:
session.run(tf.global_variables_initializer())
logi = session.run(logits, feed_dict)
logi.shape

(600, 10000)

In [31]:
First_word_output_probablity = logi[0]
First_word_output_probablity.shape

(10000,)

### Prediction
The maximum probablity

In [32]:
embedding_array= session.run(embedding, feed_dict)
np.argmax(First_word_output_probablity)

6212

So, what is the ground truth for the first word of first sentence? 

In [33]:
y[0][0]

9971

Also, you can get it from target tensor, if you want to find the embedding vector:

In [34]:
_targets

<tf.Tensor 'Placeholder_1:0' shape=(30, 20) dtype=int32>

It is time to compare logit with target

In [35]:
targ = session.run(tf.reshape(_targets, [-1]), feed_dict) 

In [36]:
first_word_target_code= targ[0]
first_word_target_code

9971

In [37]:
first_word_target_vec = session.run( tf.nn.embedding_lookup(embedding, targ[0]))
first_word_target_vec

array([ -1.04733184e-03,   2.42270827e-02,  -1.72268525e-02,
         1.20069720e-02,  -1.92369372e-02,   2.35040113e-02,
         1.19325295e-02,  -3.65709513e-03,   6.30673021e-03,
        -1.87882669e-02,   4.45487909e-03,   4.59974073e-03,
        -9.57092177e-03,   7.65682012e-03,   1.23129040e-03,
         1.18641928e-03,  -1.36597222e-02,  -1.60192177e-02,
        -2.32345592e-02,   6.34717755e-03,   1.94436572e-02,
        -8.06851871e-03,   2.55010463e-03,   8.79669562e-03,
        -1.95105486e-02,   1.68616176e-02,   2.16872953e-02,
         1.09831989e-02,  -2.17179991e-02,   8.18478316e-03,
         8.41740519e-04,   1.16804801e-03,  -9.29802656e-04,
         3.97964939e-03,   1.46834441e-02,  -1.84606202e-02,
         1.63465701e-02,   1.12980455e-02,  -6.35626912e-03,
         1.32077560e-02,  -2.02165917e-02,  -1.04783345e-02,
         3.29232961e-03,   9.09314305e-03,   3.32598388e-03,
         1.09447390e-02,  -1.85148045e-03,  -1.80765912e-02,
         5.67870215e-04,

#### Objective function

Now we want to define our objective function. Our objective is to minimize loss function, that is, to minimize the average negative log probability of the target words:

loss=−1N∑i=1Nln⁡ptargeti  
This function is already implimented and available in TensorFlow through __sequence_loss_by_example__ so we can just use it here. __sequence_loss_by_example__ is weighted cross-entropy loss for a sequence of logits (per example).  

Its arguments:  

logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].  
targets: List of 1D batch-sized int32 Tensors of the same length as logits.  
weights: List of 1D batch-sized float-Tensors of the same length as logits.  

In [38]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(_targets, [-1])],[tf.ones([batch_size * num_steps])])

loss is a 1D batch-sized float Tensor [600x1]: The log-perplexity for each sequence.

In [39]:
session.run(loss, feed_dict)

array([ 9.19458389,  9.2089529 ,  9.20383358,  9.19668484,  9.22284603,
        9.22323704,  9.21939754,  9.20577526,  9.21760178,  9.20278835,
        9.22381783,  9.22282028,  9.21599388,  9.20778465,  9.22204113,
        9.20389938,  9.20590591,  9.19645882,  9.20643234,  9.22712135,
        9.22742367,  9.20594978,  9.21986485,  9.22117615,  9.21954632,
        9.22490978,  9.22328377,  9.2225771 ,  9.20987511,  9.21405888,
        9.22658634,  9.22763443,  9.19816017,  9.20090866,  9.21199799,
        9.19970322,  9.19820786,  9.20146179,  9.19972229,  9.21308231,
        9.20467091,  9.22111702,  9.21110916,  9.22421551,  9.20310879,
        9.21938229,  9.1954031 ,  9.21675587,  9.21378136,  9.19936275,
        9.1998291 ,  9.21116543,  9.21734905,  9.20096397,  9.20203018,
        9.20098114,  9.19969749,  9.21083355,  9.21749401,  9.22101593,
        9.22098637,  9.22110558,  9.22119617,  9.22270203,  9.20559311,
        9.20961571,  9.21391582,  9.22113323,  9.2199192 ,  9.22

In [40]:
cost = tf.reduce_sum(loss) / batch_size

session.run(tf.global_variables_initializer())
session.run(cost, feed_dict)


184.21652

Now, lets store the new state as final state 

In [41]:
# 
final_state = new_state

### Training

To do gradient clipping in TensorFlow we have to take the following steps:

1. Define the optimizer.
2. Extract variables that are trainable.
3. Calculate the gradients based on the loss function.
4. Apply the optimizer to the variables / gradients tuple.

#### 1. Define Optimizer

__GradientDescentOptimizer__ constructs a new gradient descent optimizer. Later, we use constructed __optimizer__ to compute gradients for a loss and apply gradients to variables.

In [42]:
# Create a variable for the learning rate
lr = tf.Variable(0.0, trainable=False)
# Create the gradient descent optimizer with our learning rate
optimizer = tf.train.GradientDescentOptimizer(lr)


#### 2. Trainable Variables

Definining a variable, if you passed _trainable=True_, the _Variable()_ constructor automatically adds new variables to the graph collection __GraphKeys.TRAINABLE_VARIABLES__. Now, using _tf.trainable_variables()_ you can get all variables created with __trainable=True__.

In [43]:
# Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'embedding:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/weights:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/biases:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/weights:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/biases:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

we can find the name and scope of all variables:

In [44]:
tvars=tvars[3:]

In [45]:
[v.name for v in tvars]

['rnn/multi_rnn_cell/cell_1/basic_lstm_cell/weights:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/biases:0',
 'softmax_w:0',
 'softmax_b:0']

#### 3. Calculate the gradients based on the loss function

In [46]:
cost

<tf.Tensor 'truediv:0' shape=() dtype=float32>

In [47]:
tvars

[<tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/weights:0' shape=(400, 800) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/biases:0' shape=(800,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(200, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

#### Gradient:
The gradient of a function (line) is the slope of the line, or the rate of change of a function. It's a vector (a direction to move) that points in the direction of greatest increase of the function, and calculated by __derivative__ operation.

First lets recall the gradient function using an toy example:
$$ z=\left(2x^2+3xy\right)$$

In [48]:
var_x = tf.placeholder(tf.float32)
var_y = tf.placeholder(tf.float32) 
func_test = 2.0*var_x*var_x + 3.0*var_x*var_y
session.run(tf.global_variables_initializer())
feed={var_x:1.0,var_y:2.0}
session.run(func_test, feed)


8.0

The __tf.gradients()__ function allows you to compute the symbolic gradient of one tensor with respect to one or more other tensors—including variables. __tf.gradients(func,xs)__ constructs symbolic partial derivatives of sum of __func__ w.r.t. _x_ in __xs__. 

Now, lets look at the derivitive w.r.t. __var_x__:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2+3xy\right)=4x+3y $$


In [49]:
var_grad = tf.gradients(func_test, [var_x])
session.run(var_grad,feed)

[10.0]

the derivitive w.r.t. __var_y__:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2+3xy\right)=3x $$

In [50]:
var_grad = tf.gradients(func_test, [var_y])
session.run(var_grad,feed)

[3.0]

Now, we can look at gradients w.r.t all variables:

In [51]:
tf.gradients(cost, tvars)

[<tf.Tensor 'gradients_2/rnn/while/multi_rnn_cell/cell_1/basic_lstm_cell/basic_lstm_cell_1/MatMul/Enter_grad/b_acc_3:0' shape=(400, 800) dtype=float32>,
 <tf.Tensor 'gradients_2/rnn/while/multi_rnn_cell/cell_1/basic_lstm_cell/basic_lstm_cell_1/BiasAdd/Enter_grad/b_acc_3:0' shape=(800,) dtype=float32>,
 <tf.Tensor 'gradients_2/MatMul_grad/MatMul_1:0' shape=(200, 10000) dtype=float32>,
 <tf.Tensor 'gradients_2/add_grad/Reshape_1:0' shape=(10000,) dtype=float32>]

In [52]:
grad_t_list = tf.gradients(cost, tvars)
#sess.run(grad_t_list,feed_dict)




now, we have a list of tensors, t-list. We can use it to find clipped tensors. __clip_by_global_norm__ clips values of multiple tensors by the ratio of the sum of their norms.

__clip_by_global_norm__ get _t-list_ as input and returns 2 things:
 - a list of clipped tensors, so called _list_clipped_ 
 - the global norm (global_norm) of all tensors in t_list

In [53]:
max_grad_norm

5

In [54]:
# Define the gradient clipping threshold
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

[<tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_0:0' shape=(400, 800) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(800,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(200, 10000) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(10000,) dtype=float32>]

In [55]:
session.run(grads,feed_dict)

[array([[  4.00099776e-09,   2.60287625e-10,  -8.35133740e-09, ...,
          -6.60036426e-09,  -1.29708000e-09,   2.41258902e-09],
        [  3.20189830e-09,   4.92644869e-10,   6.87790003e-09, ...,
          -2.46679965e-09,  -3.85124732e-09,   2.13621587e-09],
        [ -1.34912259e-09,  -8.43564596e-09,  -1.68764220e-08, ...,
           1.46968970e-08,   1.26722366e-08,  -1.04613935e-08],
        ..., 
        [  1.67160441e-09,  -3.30584032e-10,  -8.06503420e-10, ...,
           1.06465237e-09,  -2.67157629e-09,   2.57879551e-11],
        [ -1.00444875e-09,   1.27610078e-09,   1.13445708e-09, ...,
           1.78999804e-09,   1.98703298e-09,  -1.87881954e-09],
        [  3.00192149e-10,   1.10290344e-09,   9.15292120e-10, ...,
          -4.65849359e-09,  -4.55704141e-10,   1.35898803e-09]], dtype=float32),
 array([ -1.84668181e-06,  -4.37627887e-06,  -8.41022540e-07,
         -6.67336622e-07,   5.15306374e-06,  -5.64383890e-06,
         -1.77421919e-06,  -4.13259954e-07,  -2.40099

#### 4. Apply the optimizer to the variables / gradients tuple.

In [56]:
# Create the training TensorFlow Operation through our optimizer
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [57]:
session.run(tf.global_variables_initializer())
session.run(train_op,feed_dict)

We learned how the model is build step by step. Noe, let's then create a Class that represents our model. This class needs a few things:
- We have to create the model in accordance with our defined hyperparameters
- We have to create the placeholders for our input data and expected outputs (the real data)
- We have to create the LSTM cell structure and connect them with our RNN structure
- We have to create the word embeddings and point them to the input data
- We have to create the input structure for our RNN
- We have to instanciate our RNN model and retrieve the variable in which we should expect our outputs to appear
- We need to create a logistic structure to return the probability of our words
- We need to create the loss and cost functions for our optimizer to work, and then create the optimizer
- And finally, we need to create a training operation that can be run to actually train our model


In [58]:
class PTBModel(object):

    def __init__(self, is_training):
        ######################################
        # Setting parameters for ease of use #
        ######################################
        self.batch_size = batch_size
        self.num_steps = num_steps
        size = hidden_size
        self.vocab_size = vocab_size
        
        ###############################################################################
        # Creating placeholders for our input data and expected outputs (target data) #
        ###############################################################################
        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

        ##########################################################################
        # Creating the LSTM cell structure and connect it with the RNN structure #
        ##########################################################################
        # Create the LSTM unit. 
        # This creates only the structure for the LSTM and has to be associated with a RNN unit still.
        # The argument n_hidden(size=200) of BasicLSTMCell is size of hidden layer, that is, the number of hidden units of the LSTM (inside A).
        # Size is the same as the size of our hidden layer, and no bias is added to the Forget Gate. 
        # LSTM cell processes one word at a time and computes probabilities of the possible continuations of the sentence.
        lstm_cells = []
        reuse = tf.get_variable_scope().reuse
        for _ in range(num_layers):
            cell = tf.contrib.rnn.BasicLSTMCell(size, forget_bias=0.0, reuse=reuse)
            if is_training and keep_prob < 1:
                # Unless you changed keep_prob, this won't actually execute -- this is a dropout wrapper for our LSTM unit
                # This is an optimization of the LSTM output, but is not needed at all
                cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
            lstm_cells.append(cell)
           
        # By taking in the LSTM cells as parameters, the MultiRNNCell function junctions the LSTM units to the RNN units.
        # RNN cell composed sequentially of multiple simple cells.
        stacked_lstm = tf.contrib.rnn.MultiRNNCell(lstm_cells)

        # Define the initial state, i.e., the model state for the very first data point
        # It initialize the state of the LSTM memory. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word.
        self._initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        ####################################################################
        # Creating the word embeddings and pointing them to the input data #
        ####################################################################
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            # Uses default variable initializer
            embedding = tf.get_variable("embedding", [vocab_size, size])  #[10000x200]
            # Define where to get the data for our embeddings from
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        # Unless you changed keep_prob, this won't actually execute -- this is a dropout addition for our inputs
        # This is an optimization of the input processing and is not needed at all
        if is_training and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

        ############################################
        # Creating the input structure for our RNN #
        ############################################
        # Input structure is 20x[30x200]
        # Considering each word is represended by a 200 dimentional vector, and we have 30 batchs, we create 30 word-vectors of size [30xx2000]
        #inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
        # The input structure is fed from the embeddings, which are filled in by the input data
        # Feeding a batch of b sentences to a RNN:
        # In step 1,  first word of each of the b sentences (in a batch) is input in parallel.  
        # In step 2,  second word of each of the b sentences is input in parallel. 
        # The parallelism is only for efficiency.  
        # Each sentence in a batch is handled in parallel, but the network sees one word of a sentence at a time and does the computations accordingly. 
        # All the computations involving the words of all sentences in a batch at a given time step are done in parallel. 

        ####################################################################################################
        # Instanciating our RNN model and retrieving the structure for returning the outputs and the state #
        ####################################################################################################
        outputs, state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=self._initial_state)

        #########################################################################
        # Creating a logistic unit to return the probability of the output word #
        #########################################################################
        output = tf.reshape(outputs, [-1, size])
        softmax_w = tf.get_variable("softmax_w", [size, vocab_size]) #[200x1000]
        softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
        logits = tf.matmul(output, softmax_w) + softmax_b

        #########################################################################
        # Defining the loss and cost functions for the model's learning to work #
        #########################################################################
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(self._targets, [-1])],
                                                      [tf.ones([batch_size * num_steps])])
        self._cost = cost = tf.reduce_sum(loss) / batch_size

        # Store the final state
        self._final_state = state

        #Everything after this point is relevant only for training
        if not is_training:
            return

        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = tf.trainable_variables()
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), max_grad_norm)
        # Create the gradient descent optimizer with our learning rate
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        # Create the training TensorFlow Operation through our optimizer
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))

    # Helper functions for our LSTM RNN class

    # Assign the learning rate for this model
    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

    # Returns the input data for this model at a point in time
    @property
    def input_data(self):
        return self._input_data

    # Returns the targets for this model at a point in time
    @property
    def targets(self):
        return self._targets

    # Returns the initial state for this model
    @property
    def initial_state(self):
        return self._initial_state

    # Returns the defined Cost
    @property
    def cost(self):
        return self._cost

    # Returns the final state for this model
    @property
    def final_state(self):
        return self._final_state

    # Returns the current learning rate for this model
    @property
    def lr(self):
        return self._lr

    # Returns the training operation defined for this model
    @property
    def train_op(self):
        return self._train_op

With that, the actual structure of our Recurrent Neural Network with Long Short-Term Memory is finished. What remains for us to do is to actually create the methods to run through time -- that is, the `run_epoch` method to be run at each epoch and a `main` script which ties all of this together.

What our `run_epoch` method should do is take our input data and feed it to the relevant operations. This will return at the very least the current result for the cost function.

In [62]:
##########################################################################################################################
# run_epoch takes as parameters the current session, the model instance, the data to be fed, and the operation to be run #
##########################################################################################################################
def run_epoch(session, m, data, eval_op, verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0
    #state = m.initial_state.eval()
    #m.initial_state = tf.convert_to_tensor(m.initial_state) 
    #state = m.initial_state.eval()
    state = session.run(m.initial_state)
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
 
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        cost, state, _ = session.run([m.cost, m.final_state, eval_op],
                                     {m.input_data: x,
                                      m.targets: y,
                                      m.initial_state: state})
        
        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += cost
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and (step % 10) == 0:
            print("({:.2%})    Perplexity={:.3f}    Speed={:.0f} wps".format(
                    step * 1.0 / epoch_size, 
                    np.exp(costs / iters), 
                    iters * m.batch_size / (time.time() - start_time))
                 )

    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)


Now, we create the `main` method to tie everything together. The code here reads the data from the directory, using the `reader` helper module, and then trains and evaluates the model on both a testing and a validating subset of data.

In [60]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _ = raw_data

In [70]:
#Initializes the Execution Graph and the Session
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-init_scale,init_scale)
    
    # Instantiates the model for training
    # tf.variable_scope add a prefix to the variables created with tf.get_variable
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(is_training=True)
        
    # Reuses the trained parameters for the validation and testing models
    # They are different instances but use the same variables for weights and biases, 
    # they just don't change when data is input
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(is_training=False)
        mtest = PTBModel(is_training=False)

    #Initialize all variables
    tf.global_variables_initializer().run()
    
    # Set initial learning rate
    m.assign_lr(session=session, lr_value=learning_rate)
    
    for i in range(max_max_epoch):
        print("Epoch %d : Learning rate: %.3f" % (i + 1, session.run(m.lr)))
               
        # Run the loop for this epoch in the training model
        train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                   verbose=True)
        print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        # Run the loop for this epoch in the validation model
        valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
        
        # Define the decay for the next epoch 
        lr_decay = decay * ((max_max_epoch - i) / max_max_epoch)
        
        # Set the decayed learning rate as the learning rate for the next epoch
        m.assign_lr(session, learning_rate * lr_decay)
    
    # Run the loop in the testing model to see how effective was our training
    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    
    print("Test Perplexity: %.3f" % test_perplexity)

Epoch 1 : Learning rate: 1.000
(0.00%)    Perplexity=10015.681    Speed=2745 wps
(0.65%)    Perplexity=5668.811    Speed=4017 wps
(1.29%)    Perplexity=3186.960    Speed=4048 wps
(1.94%)    Perplexity=2392.133    Speed=4084 wps
(2.58%)    Perplexity=2035.965    Speed=4104 wps
(3.23%)    Perplexity=1799.039    Speed=4122 wps
(3.87%)    Perplexity=1649.212    Speed=4137 wps
(4.52%)    Perplexity=1520.943    Speed=4148 wps
(5.16%)    Perplexity=1424.213    Speed=4157 wps
(5.81%)    Perplexity=1362.012    Speed=4163 wps
(6.46%)    Perplexity=1305.276    Speed=4166 wps
(7.10%)    Perplexity=1253.879    Speed=4171 wps
(7.75%)    Perplexity=1214.834    Speed=4174 wps
(8.39%)    Perplexity=1175.636    Speed=4177 wps
(9.04%)    Perplexity=1144.073    Speed=4179 wps
(9.68%)    Perplexity=1118.170    Speed=4181 wps
(10.33%)    Perplexity=1082.403    Speed=4183 wps
(10.97%)    Perplexity=1059.396    Speed=4168 wps
(11.62%)    Perplexity=1033.964    Speed=4170 wps
(12.27%)    Perplexity=1010.451   

(6.46%)    Perplexity=175.068    Speed=3733 wps
(7.10%)    Perplexity=176.832    Speed=3774 wps
(7.75%)    Perplexity=178.631    Speed=3807 wps
(8.39%)    Perplexity=178.417    Speed=3836 wps
(9.04%)    Perplexity=178.469    Speed=3766 wps
(9.68%)    Perplexity=179.058    Speed=3674 wps
(10.33%)    Perplexity=178.738    Speed=3678 wps
(10.97%)    Perplexity=179.885    Speed=3690 wps
(11.62%)    Perplexity=179.845    Speed=3705 wps
(12.27%)    Perplexity=179.013    Speed=3729 wps
(12.91%)    Perplexity=177.816    Speed=3751 wps
(13.56%)    Perplexity=176.914    Speed=3765 wps
(14.20%)    Perplexity=175.427    Speed=3772 wps
(14.85%)    Perplexity=175.368    Speed=3778 wps
(15.49%)    Perplexity=174.418    Speed=3793 wps
(16.14%)    Perplexity=174.443    Speed=3808 wps
(16.79%)    Perplexity=174.490    Speed=3822 wps
(17.43%)    Perplexity=174.097    Speed=3826 wps
(18.08%)    Perplexity=173.453    Speed=3830 wps
(18.72%)    Perplexity=173.827    Speed=3837 wps
(19.37%)    Perplexity=173

(13.56%)    Perplexity=130.713    Speed=3764 wps
(14.20%)    Perplexity=129.844    Speed=3725 wps
(14.85%)    Perplexity=129.943    Speed=3667 wps
(15.49%)    Perplexity=129.432    Speed=3667 wps
(16.14%)    Perplexity=129.510    Speed=3676 wps
(16.79%)    Perplexity=129.795    Speed=3687 wps
(17.43%)    Perplexity=129.669    Speed=3704 wps
(18.08%)    Perplexity=129.246    Speed=3719 wps
(18.72%)    Perplexity=129.689    Speed=3730 wps
(19.37%)    Perplexity=129.695    Speed=3736 wps
(20.01%)    Perplexity=129.531    Speed=3736 wps
(20.66%)    Perplexity=128.593    Speed=3669 wps
(21.30%)    Perplexity=128.146    Speed=3632 wps
(21.95%)    Perplexity=127.622    Speed=3647 wps
(22.60%)    Perplexity=126.989    Speed=3660 wps
(23.24%)    Perplexity=126.812    Speed=3674 wps
(23.89%)    Perplexity=127.061    Speed=3686 wps
(24.53%)    Perplexity=127.232    Speed=3698 wps
(25.18%)    Perplexity=126.868    Speed=3673 wps
(25.82%)    Perplexity=126.385    Speed=3641 wps
(26.47%)    Perplexi

(20.66%)    Perplexity=105.238    Speed=3748 wps
(21.30%)    Perplexity=104.889    Speed=3715 wps
(21.95%)    Perplexity=104.480    Speed=3729 wps
(22.60%)    Perplexity=103.995    Speed=3742 wps
(23.24%)    Perplexity=103.840    Speed=3755 wps
(23.89%)    Perplexity=104.101    Speed=3765 wps
(24.53%)    Perplexity=104.256    Speed=3775 wps
(25.18%)    Perplexity=104.007    Speed=3758 wps
(25.82%)    Perplexity=103.673    Speed=3724 wps
(26.47%)    Perplexity=103.614    Speed=3722 wps
(27.11%)    Perplexity=103.217    Speed=3733 wps
(27.76%)    Perplexity=102.995    Speed=3744 wps
(28.41%)    Perplexity=102.780    Speed=3754 wps
(29.05%)    Perplexity=102.826    Speed=3760 wps
(29.70%)    Perplexity=102.445    Speed=3761 wps
(30.34%)    Perplexity=102.049    Speed=3731 wps
(30.99%)    Perplexity=101.807    Speed=3713 wps
(31.63%)    Perplexity=101.656    Speed=3717 wps
(32.28%)    Perplexity=101.336    Speed=3721 wps
(32.92%)    Perplexity=101.262    Speed=3702 wps
(33.57%)    Perplexi

(29.70%)    Perplexity=88.116    Speed=3654 wps
(30.34%)    Perplexity=87.772    Speed=3663 wps
(30.99%)    Perplexity=87.551    Speed=3670 wps
(31.63%)    Perplexity=87.439    Speed=3675 wps
(32.28%)    Perplexity=87.174    Speed=3681 wps
(32.92%)    Perplexity=87.121    Speed=3648 wps
(33.57%)    Perplexity=86.789    Speed=3619 wps
(34.22%)    Perplexity=86.646    Speed=3630 wps
(34.86%)    Perplexity=86.327    Speed=3640 wps
(35.51%)    Perplexity=85.858    Speed=3649 wps
(36.15%)    Perplexity=85.240    Speed=3659 wps
(36.80%)    Perplexity=84.732    Speed=3669 wps
(37.44%)    Perplexity=84.729    Speed=3661 wps
(38.09%)    Perplexity=84.633    Speed=3639 wps
(38.73%)    Perplexity=84.560    Speed=3640 wps
(39.38%)    Perplexity=84.572    Speed=3649 wps
(40.03%)    Perplexity=84.577    Speed=3657 wps
(40.67%)    Perplexity=84.459    Speed=3666 wps
(41.32%)    Perplexity=84.430    Speed=3674 wps
(41.96%)    Perplexity=84.282    Speed=3682 wps
(42.61%)    Perplexity=84.177    Speed=3

...

(56.81%)    Perplexity=61.406    Speed=3692 wps
(57.46%)    Perplexity=61.343    Speed=3683 wps
(58.10%)    Perplexity=61.271    Speed=3658 wps
(58.75%)    Perplexity=61.067    Speed=3660 wps
(59.39%)    Perplexity=60.864    Speed=3666 wps
(60.04%)    Perplexity=60.722    Speed=3672 wps
(60.68%)    Perplexity=60.614    Speed=3678 wps
(61.33%)    Perplexity=60.528    Speed=3683 wps
(61.98%)    Perplexity=60.340    Speed=3686 wps
(62.62%)    Perplexity=60.188    Speed=3673 wps
(63.27%)    Perplexity=60.146    Speed=3666 wps
(63.91%)    Perplexity=60.090    Speed=3671 wps
(64.56%)    Perplexity=59.980    Speed=3677 wps
(65.20%)    Perplexity=59.875    Speed=3682 wps
(65.85%)    Perplexity=59.956    Speed=3688 wps
(66.49%)    Perplexity=59.975    Speed=3693 wps
(67.14%)    Perplexity=60.021    Speed=3688 wps
(67.79%)    Perplexity=60.061    Speed=3676 wps
(68.43%)    Perplexity=60.017    Speed=3676 wps
(69.08%)    Perplexity=60.001    Speed=3681 wps
(69.72%)    Perplexity=60.044    Speed=3

(65.85%)    Perplexity=55.290    Speed=3735 wps
(66.49%)    Perplexity=55.309    Speed=3736 wps
(67.14%)    Perplexity=55.351    Speed=3739 wps
(67.79%)    Perplexity=55.386    Speed=3742 wps
(68.43%)    Perplexity=55.342    Speed=3746 wps
(69.08%)    Perplexity=55.327    Speed=3750 wps
(69.72%)    Perplexity=55.366    Speed=3754 wps
(70.37%)    Perplexity=55.327    Speed=3755 wps
(71.01%)    Perplexity=55.312    Speed=3757 wps
(71.66%)    Perplexity=55.369    Speed=3741 wps
(72.30%)    Perplexity=55.299    Speed=3726 wps
(72.95%)    Perplexity=55.222    Speed=3731 wps
(73.60%)    Perplexity=55.110    Speed=3735 wps
(74.24%)    Perplexity=55.008    Speed=3740 wps
(74.89%)    Perplexity=54.910    Speed=3744 wps
(75.53%)    Perplexity=54.884    Speed=3748 wps
(76.18%)    Perplexity=54.817    Speed=3743 wps
(76.82%)    Perplexity=54.868    Speed=3732 wps
(77.47%)    Perplexity=54.886    Speed=3730 wps
(78.11%)    Perplexity=54.918    Speed=3732 wps
(78.76%)    Perplexity=54.901    Speed=3

(74.89%)    Perplexity=51.092    Speed=3709 wps
(75.53%)    Perplexity=51.067    Speed=3712 wps
(76.18%)    Perplexity=51.003    Speed=3717 wps
(76.82%)    Perplexity=51.049    Speed=3721 wps
(77.47%)    Perplexity=51.064    Speed=3724 wps
(78.11%)    Perplexity=51.090    Speed=3726 wps
(78.76%)    Perplexity=51.072    Speed=3726 wps
(79.41%)    Perplexity=51.111    Speed=3707 wps
(80.05%)    Perplexity=51.123    Speed=3701 wps
(80.70%)    Perplexity=51.145    Speed=3705 wps
(81.34%)    Perplexity=51.159    Speed=3709 wps
(81.99%)    Perplexity=51.114    Speed=3713 wps
(82.63%)    Perplexity=51.094    Speed=3717 wps
(83.28%)    Perplexity=51.036    Speed=3721 wps
(83.93%)    Perplexity=50.899    Speed=3713 wps
(84.57%)    Perplexity=50.776    Speed=3703 wps
(85.22%)    Perplexity=50.597    Speed=3704 wps
(85.86%)    Perplexity=50.483    Speed=3706 wps
(86.51%)    Perplexity=50.359    Speed=3709 wps
(87.15%)    Perplexity=50.258    Speed=3713 wps
(87.80%)    Perplexity=50.208    Speed=3

(83.93%)    Perplexity=47.746    Speed=3734 wps
(84.57%)    Perplexity=47.629    Speed=3737 wps
(85.22%)    Perplexity=47.461    Speed=3741 wps
(85.86%)    Perplexity=47.353    Speed=3744 wps
(86.51%)    Perplexity=47.236    Speed=3746 wps
(87.15%)    Perplexity=47.140    Speed=3747 wps
(87.80%)    Perplexity=47.091    Speed=3749 wps
(88.44%)    Perplexity=47.044    Speed=3752 wps
(89.09%)    Perplexity=47.053    Speed=3756 wps
(89.74%)    Perplexity=47.106    Speed=3758 wps
(90.38%)    Perplexity=47.078    Speed=3760 wps
(91.03%)    Perplexity=47.076    Speed=3761 wps
(91.67%)    Perplexity=47.001    Speed=3749 wps
(92.32%)    Perplexity=46.959    Speed=3736 wps
(92.96%)    Perplexity=46.928    Speed=3740 wps
(93.61%)    Perplexity=46.831    Speed=3743 wps
(94.25%)    Perplexity=46.778    Speed=3746 wps
(94.90%)    Perplexity=46.716    Speed=3749 wps
(95.55%)    Perplexity=46.677    Speed=3752 wps
(96.19%)    Perplexity=46.641    Speed=3748 wps
(96.84%)    Perplexity=46.659    Speed=3

(92.96%)    Perplexity=44.454    Speed=3711 wps
(93.61%)    Perplexity=44.360    Speed=3709 wps
(94.25%)    Perplexity=44.309    Speed=3711 wps
(94.90%)    Perplexity=44.250    Speed=3712 wps
(95.55%)    Perplexity=44.212    Speed=3716 wps
(96.19%)    Perplexity=44.175    Speed=3719 wps
(96.84%)    Perplexity=44.187    Speed=3721 wps
(97.48%)    Perplexity=44.172    Speed=3723 wps
(98.13%)    Perplexity=44.176    Speed=3724 wps
(98.77%)    Perplexity=44.152    Speed=3716 wps
(99.42%)    Perplexity=44.160    Speed=3702 wps
Epoch 12 : Train Perplexity: 44.165
Epoch 12 : Valid Perplexity: 139.006
Epoch 13 : Learning rate: 0.077
(0.00%)    Perplexity=77.454    Speed=2969 wps
(0.65%)    Perplexity=52.123    Speed=3014 wps
(1.29%)    Perplexity=50.311    Speed=2872 wps
(1.94%)    Perplexity=47.490    Speed=3114 wps
(2.58%)    Perplexity=45.689    Speed=3280 wps
(3.23%)    Perplexity=46.339    Speed=3404 wps
(3.87%)    Perplexity=45.920    Speed=3518 wps
(4.52%)    Perplexity=45.436    Speed=

As you can see, the model's perplexity rating drops very quickly after a few iterations. As was elaborated before, **lower Perplexity means that the model is more certain about its prediction**. As such, we can be sure that this model is performing well!

-------

This is the end of the **Applying Recurrent Neural Networks to Text Processing** notebook. Hopefully you now have a better understanding of Recurrent Neural Networks and how to implement one utilizing TensorFlow. Thank you for reading this notebook, and good luck on your studies.

Created by <a href="https://br.linkedin.com/in/walter-gomes-de-amorim-junior-624726121">Walter Gomes de Amorim Junior</a>, <a href = "https://linkedin.com/in/saeedaghabozorgi"> Saeed Aghabozorgi </a></h4>